Import libraries and extensions

In [1]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options=tf.compat.v1.GPUOptions(allow_growth=True))
sess = tf.compat.v1.Session(config=config)
import numpy as np
import os
import tensorflow
import matplotlib.pyplot as plt
import datetime
import pickle
import pandas as pd

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import tensorflow as tf
from tensorflow import keras
from tensorflow import math
print(tf.__version__)
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.layers import Dense,GlobalMaxPooling2D,MaxPool2D
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.models import load_model

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers

2.5.0


Put the full directories of each local model into a list 'institutions', to easily access them all later on.

In [15]:
directory = r'C:\Documents\initial\weights'
save_dir = r'C:\Documents\initial'

institutions = []

for i in os.listdir(directory):
    institutions.append(os.path.join(directory,i))


Function to construct model, used so the local model can be compiled and loaded in, and its weights can be accessed.

In [4]:
def create_model(input_shape):

    base_model=VGG16(weights=None,include_top=False,input_shape=input_shape)
    x=base_model.output
    x=keras.layers.GlobalMaxPooling2D()(x)
    x=keras.layers.Dense(1024,activation='relu')(x) 
    x=keras.layers.Dense(1024,activation='relu')(x)
    x=keras.layers.Dropout(0.25)(x)
    x=keras.layers.Dense(1024,activation='relu')(x) 
    x=keras.layers.Dense(512,activation='relu')(x) 
    preds=keras.layers.Dense(1,activation='sigmoid')(x)
    
    model=keras.Model(inputs=base_model.input,outputs=preds)
    model.trainable = True

    for layer in model.layers[:16]:
        layer.trainable=False
    for layer in model.layers[16:]:
        layer.trainable=True

    return model

Construct an array whose shape matches that of the local models' weights array, to serve as the global model.

In [5]:
def build_fed(fed_shape):
    fed = []

    input_shape = (224, 224, 3)
    for i in range(len(fed_shape.layers)):
        f_layer = []
        w = fed_shape.layers[i].get_weights()
        for j in range(len(w)):
            f_layer.append(np.empty(np.shape(w[j])))
        fed.append(f_layer)
    return fed

Add up the weights element-wise, putting the results in the respective coordinates of the global model.

In [6]:
def fed_total(fed, institutions):
    #for each institution model
    for mod in institutions:
        #load in the model
        model = create_model(input_shape)
        model.load_weights(mod)
        #for each layer in the model
        for l in range(len(model.layers)):
            #get the weights array of this layer
            w = model.layers[l].get_weights()
            #for each dimension in the weight array
            for i in range(len(w)):
                #add the current institution's layer weights array to the respective
                #layer weights array already in the global model
                fed[l][i] = np.add(fed[l][i], w[i])   
    return fed

Average the results by dividing the weights from fed_total by how many institutions were used in the federation. The function goes through each layer from the fed_total array, and accounts for the pooling layers which remain empty.

In [7]:
def fed_avg(fed):
    # save averaged layers into avg[] array
    avg = []
    # for every layer in the model
    for l in range(len(fed)):
        # save averaged weights into avg_layer[] array
        avg_layer = []
        if len(fed[l]) > 0:
            # for each weight in the current layer
            for i in range(len(fed[l])):
                # divide each total weight to get avg
                # putting the result in the avg_layer[] array
                avg_layer.append((fed[l][i])/len(institutions))
            # put the averaged layer into the avg[] array
            avg.append(avg_layer)
        else:
            avg.append(fed[l])
    return avg


Convert the averaged layers array into a proper TensorFlow(TF) model.

In [8]:
def set_fed_mod(fed_shape, avg):
    for i in range(len(fed_shape.layers)):
        layer = fed_shape.layers[i]
        layer.set_weights(avg[i])
    return fed_shape

Call the functions to declare the model shape, initialize an empty federation model, total the weights from each institution element-wise, average the totalled weights, and save the averaged weights array into a TF model.

In [10]:
input_shape = (224, 224, 3)

fed_shape = create_model(input_shape)
fed = build_fed(fed_shape)
total = fed_total(fed, institutions)
avg = fed_avg(total)
global_model = set_fed_mod(fed_shape, avg)

Save the global model.

In [59]:
weights_save_dir = os.path.join(save_dir, 'Global_X.h5')
global_model.save_weights(weights_save_dir);

In [63]:
fed_shape.summary()

Model: "model_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0  

Check that the federated global model shape matches the local model shape by comparing the sizes of each layer from both models.

In [74]:
model_check = create_model(input_shape)

In [75]:
#fed is the federated layers list
#model is the institutions[0] model layers list

def arr_shape(mod, fed):
    for l in range(len(mod.layers)):
        weights = (mod.layers[l].get_weights())
        print(mod.layers[l].name)
        for i in range(len(weights)):
            print(i)
            print('local shape:', np.shape(weights[i]))
            print('global shape:', np.shape(fed[l][i]))
        print('model layer len:', len(weights))
        print('global layer len:', len(fed[l]))
        print('\n')


In [73]:
arr_shape(model_check, avg)

input_29
model layer len: 0
global layer len: 0


block1_conv1
0
local shape: (3, 3, 3, 64)
global shape: (3, 3, 3, 64)
1
local shape: (64,)
global shape: (64,)
model layer len: 2
global layer len: 2


block1_conv2
0
local shape: (3, 3, 64, 64)
global shape: (3, 3, 64, 64)
1
local shape: (64,)
global shape: (64,)
model layer len: 2
global layer len: 2


block1_pool
model layer len: 0
global layer len: 0


block2_conv1
0
local shape: (3, 3, 64, 128)
global shape: (3, 3, 64, 128)
1
local shape: (128,)
global shape: (128,)
model layer len: 2
global layer len: 2


block2_conv2
0
local shape: (3, 3, 128, 128)
global shape: (3, 3, 128, 128)
1
local shape: (128,)
global shape: (128,)
model layer len: 2
global layer len: 2


block2_pool
model layer len: 0
global layer len: 0


block3_conv1
0
local shape: (3, 3, 128, 256)
global shape: (3, 3, 128, 256)
1
local shape: (256,)
global shape: (256,)
model layer len: 2
global layer len: 2


block3_conv2
0
local shape: (3, 3, 256, 256)
global shape: (